# What is Named Entity Recognition ? 

Es un problema de *Sequence Labeling*.  A diferencia de obtener una clase para todo un texto, cada token del texto debe ser taggeado.

### Algo de teoría breve

#### Sequence Labeling & Structured Output

Al ser un proceso de etiquetar sequencias, que puede ser mejor, etiquetar toda la sequencia de golpe, o ir palabra a palabra?

Bueno, con lo que hemos visto hasta ahora, tanto en Machine Learning, como en Deep Learning, no hay manera de etiquetar todo de golpe, porque normalmente tenemos datasets tal que las *ys* dependen unicamente de un set de features representado por *x*. En NLP muchos de estos problemas no tienen sentido, tal y como hemos visto en Language Modeling, hay casos en los que la palabra actual depende, tanto de lo que hemos dicho(pasado), como de lo que vamos a decir (futuro). De hecho, cuando vamos a hablar o escribir, *normalmente y no todos* pensamos que vamos a decir, es decir, hacemos una predicción de aquello futuro, y luego vamos articulando esa idea (pasado). Para estos casos, hay modelos probabilísticos como los modelos markovianos de variables latentes (Hidden Markov Models [HMM](https://en.wikipedia.org/wiki/Hidden_Markov_model)), que quedan fuera del temario del curso, o Conditional Random Fields ([CRF](https://en.wikipedia.org/wiki/Conditional_random_field)) como su alternativa más usada para problemas de este estilo. 

Este último, y solo a modo de que os suene, cae en la categoría de *structured prediction* ([wiki](https://en.wikipedia.org/wiki/Structured_prediction)). Structured prediction no es ni más ni menos que lo que nosotros querríamos para este tipo de productos, es decir, en lugar de que *y* este condicionada sólo a una serie de *features x*, este condicionado a todo el output. Y que todo este entrelazado, que por ejemplo un tag posterior, pueda afectar a uno anterior y viceversa.

![](https://i.imgur.com/ukAr3Uh.jpg)

Entonces si no vemos esto dentro del curso, hemos terminado la clase porque no podemos resolver el problema. Bueno no del todo.

Como siempre, hay alternativas, que quizás no cumplen con todos los requisitos que querriamos pero dan resultados correctos, sobretodo para ver baselines.

Entonces, si no podemos predecir algo en base al tiempo, haremos que el tiempo sea una feature. Es decir, en una feature meteremos tambien las features de *t-n* hasta *t+n*. Es decir, que la feature *x_n* estará compuesta por los la concatenación de elementos de features pasadas, y features futuras, y haremos que el clasificador tenga suficiente información en cada pareja *x,y*. Veremos en breves como se componen estas features.

![](https://i.imgur.com/Wail7yI.jpg =400x)
![](https://i.imgur.com/irjqooW.jpg =400x)

De esta forma, conseguimos crear features "independientes" del tiempo y podremos entrenar como siempre.

### Named Entity Recognition

El Objectivo es asignar doble-tag uno para cada clase, persona, organizacion, lo que queramos, y el otro es el conocido como BIO.

*   Begin: Inicio de una entidad
*   Inside: Dentro de una entidad
*   Outside: No es una entida

Actualmente y bastantes taggers correctos, pero en general, hay que adaptarlo al dominio. Un caso muy claro, es el de esos taggers, como el que veremos que se basa entre otras features, en si una palabra esta en mayúsculas. En un dominio más casual, como twitter, o cualquier otro ambito del tipo de internet.

![](https://i.imgur.com/a8Zregf.jpg)

Cual es la idea para entrenar? Para todo un texto usamos n-grams fomo features normalmente, entre otras. Aqui la idea es palabra por palabra poder decidir que tag tiene. Se definen una seria de features, que no son n-grams. Ahoar veremos un subset de estas features. Para lo demas, sera como cualquier otro modelo. Representar como input las features, tener preparado para cada input, un output, escoger clasificador y función de objetivo y a entrenar!

### Imports

In [0]:
import csv
from tqdm import tqdm
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])

# Building sets of features

Hasta ahora, hemos usado como features solo una minúscula parte del potencial linguístico que tienen los textos. Para este ejercicio entrenaremos con nuevas features, algunas muy obvias, otras quizas no tanto. En la imagen se pueden ver features para crear un Part of Speech tagger, nosotros lo adaptaremos a un Named Entity Recognizer.

![alt text](https://i.imgur.com/1SAC0TU.jpg =500x)

From Neural Network Methods for NLP by Yoav Goldberg


Acordemonos que las features hay que adaptarlas no solo al problema, sino al dataset. El problema nos puede dar un estimado de que features usar, pero como siempre, nosotros podemos generar, quitar o combinar features como queramos. Por ejemplo, en el case de Entity Recognizers, que una palabra este en mayúsculas quizás es muy relevante, pero si estamos en un contexto de internet, donde la gente no sigue una convención fija de escriptura, es menos relevante.

Para que veais un ejemplo de la complejidad de la tarea.




> Mal
![](https://i.imgur.com/wCOsdsk.png)
![](https://i.imgur.com/EO89nLZ.png)

> Bien
![](https://i.imgur.com/amsQr9T.png)

In [0]:
import re
 
def shape(word):
    word_shape = 'other'
    if re.match('[0-9]+(\.[0-9]*)?|[0-9]*\.[0-9]+$', word):
        word_shape = 'number'
    elif re.match('\W+$', word):
        word_shape = 'punct'
    elif re.match('[A-Z][a-z]+$', word):
        word_shape = 'capitalized'
    elif re.match('[A-Z]+$', word):
        word_shape = 'uppercase'
    elif re.match('[a-z]+$', word):
        word_shape = 'lowercase'
    elif re.match('[A-Z][a-z]+[A-Z][a-z]+[A-Za-z]*$', word):
        word_shape = 'camelcase'
    elif re.match('[A-Za-z]+$', word):
        word_shape = 'mixedcase'
    elif re.match('__.+__$', word):
        word_shape = 'wildcard'
    elif re.match('[A-Za-z0-9]+\.$', word):
        word_shape = 'ending-dot'
    elif re.match('[A-Za-z0-9]+\.[A-Za-z0-9\.]+\.$', word):
        word_shape = 'abbreviation'
    elif re.match('[A-Za-z0-9]+\-[A-Za-z0-9\-]+.*$', word):
        word_shape = 'contains-hyphen'
 
    return word_shape

In [0]:
from sklearn.linear_model import Perceptron
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline